### Test Spotify API

In [19]:
from supabase import create_client
import os
import dotenv

dotenv.load_dotenv()

supabase_url = os.getenv("SUPABASE_URL")
supabase_public_anon_key = os.getenv("SUPABASE_PUBLIC_ANON_KEY")

supabase = create_client(supabase_url, supabase_public_anon_key)

# --- Sign in user (must exist in Supabase Auth) ---
email = "darragh@gmail.com"
password = "starwars111"

response = supabase.auth.sign_in_with_password({"email": email, "password": password})

# --- Extract access token ---
token = response.session.access_token
print("JWT Token:\n",token)


JWT Token:
 eyJhbGciOiJFUzI1NiIsImtpZCI6IjU3OWEwODNiLWFjMTMtNDQ2OC1iMTRmLTM5Y2QyZDc1YjhiZSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJodHRwczovL3diYmN1dXZveGdtdGxxdWtidXp2LnN1cGFiYXNlLmNvL2F1dGgvdjEiLCJzdWIiOiJiZTE4OTA4Yy04NzJmLTQ1MmEtYjg3Yi00NTJhMzQ2N2E2MGIiLCJhdWQiOiJhdXRoZW50aWNhdGVkIiwiZXhwIjoxNzYxNDg3NzU0LCJpYXQiOjE3NjE0ODQxNTQsImVtYWlsIjoiZGFycmFnaEBnbWFpbC5jb20iLCJwaG9uZSI6IiIsImFwcF9tZXRhZGF0YSI6eyJwcm92aWRlciI6ImVtYWlsIiwicHJvdmlkZXJzIjpbImVtYWlsIl19LCJ1c2VyX21ldGFkYXRhIjp7ImVtYWlsX3ZlcmlmaWVkIjp0cnVlfSwicm9sZSI6ImF1dGhlbnRpY2F0ZWQiLCJhYWwiOiJhYWwxIiwiYW1yIjpbeyJtZXRob2QiOiJwYXNzd29yZCIsInRpbWVzdGFtcCI6MTc2MTQ4NDE1NH1dLCJzZXNzaW9uX2lkIjoiNGEzNmI1MDgtNGNhMS00NzQ0LWI1ZTktMWE1NWY3NjFiN2M3IiwiaXNfYW5vbnltb3VzIjpmYWxzZX0.yLfKbo7hUYGldNlyaq3FkZgM7YRgmTAyVnf0Ijt6mMbaD06KqcuZJgz0-npuX4mqPNQo6tH54tuBD44rzj2bQA


In [23]:
# --- Imports ---
from supabase import create_client
import os
import jwt
import requests
from jwt.algorithms import ECAlgorithm
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_PUBLIC_ANON_KEY = os.getenv("SUPABASE_PUBLIC_ANON_KEY")

# --- Create Supabase client ---
supabase = create_client(SUPABASE_URL, SUPABASE_PUBLIC_ANON_KEY)

# --- Sign in an existing user ---
email = "darragh@gmail.com"
password = "starwars111"

response = supabase.auth.sign_in_with_password({"email": email, "password": password})
token = response.session.access_token
print("✅ JWT Token:\n", token)

# --- Fetch Supabase JWKs (public keys) ---
JWK_URL = f"{SUPABASE_URL}/auth/v1/.well-known/jwks.json"
jwks = requests.get(JWK_URL).json()

# --- Find the key matching the token's kid ---
header = jwt.get_unverified_header(token)
matching_key = next(k for k in jwks["keys"] if k["kid"] == header["kid"])

# --- Convert JWK to public key ---
public_key = ECAlgorithm.from_jwk(matching_key)

# --- Verify and decode JWT (ES256) ---
decoded = jwt.decode(token, public_key, algorithms=["ES256"], audience="authenticated")
print("\n✅ Token verified successfully. Decoded payload:")
print(decoded)


✅ JWT Token:
 eyJhbGciOiJFUzI1NiIsImtpZCI6IjU3OWEwODNiLWFjMTMtNDQ2OC1iMTRmLTM5Y2QyZDc1YjhiZSIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJodHRwczovL3diYmN1dXZveGdtdGxxdWtidXp2LnN1cGFiYXNlLmNvL2F1dGgvdjEiLCJzdWIiOiJiZTE4OTA4Yy04NzJmLTQ1MmEtYjg3Yi00NTJhMzQ2N2E2MGIiLCJhdWQiOiJhdXRoZW50aWNhdGVkIiwiZXhwIjoxNzYxNDg3ODQ2LCJpYXQiOjE3NjE0ODQyNDYsImVtYWlsIjoiZGFycmFnaEBnbWFpbC5jb20iLCJwaG9uZSI6IiIsImFwcF9tZXRhZGF0YSI6eyJwcm92aWRlciI6ImVtYWlsIiwicHJvdmlkZXJzIjpbImVtYWlsIl19LCJ1c2VyX21ldGFkYXRhIjp7ImVtYWlsX3ZlcmlmaWVkIjp0cnVlfSwicm9sZSI6ImF1dGhlbnRpY2F0ZWQiLCJhYWwiOiJhYWwxIiwiYW1yIjpbeyJtZXRob2QiOiJwYXNzd29yZCIsInRpbWVzdGFtcCI6MTc2MTQ4NDI0Nn1dLCJzZXNzaW9uX2lkIjoiMWJlMjJlZmMtNzQ2ZS00M2EzLTlhY2YtMGFmYTk5YzM2NGU0IiwiaXNfYW5vbnltb3VzIjpmYWxzZX0.ewRIPHrjwv8ku9rsSBQGqsCwHHARRh8-h_Rn-Voas3LvqfG3gkD0o1nmOMgc35MvC1apu-ehN7zDyfP0VeAM_w

✅ Token verified successfully. Decoded payload:
{'iss': 'https://wbbcuuvoxgmtlqukbuzv.supabase.co/auth/v1', 'sub': 'be18908c-872f-452a-b87b-452a3467a60b', 'aud': 'authenticated', 'exp': 176

In [10]:
import requests
import json
import os
from dotenv import load_dotenv
import base64

load_dotenv()




def get_spotify_data():


    # Encode client credentials
    auth_str = f"{os.getenv('CLIENT_ID')}:{os.getenv('CLIENT_SECRET')}"
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()

    # Request token
    response = requests.post(
        "https://accounts.spotify.com/api/token",
        headers={"Authorization": f"Basic {b64_auth_str}"},
        data={"grant_type": "client_credentials"},
    )

    ACCESS_TOKEN = response.json()["access_token"]
    print("Access Token:", ACCESS_TOKEN)

    
    url = "https://api.spotify.com/v1/search"


    params = {
        "q": "greedy",
        "type": "track,artist,album",
        "limit": 5
    }

    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }

    # Send GET request
    response = requests.get(url, headers=headers, params=params)

    # Check if request was successful
    if response.status_code == 200:
        data = response.json()
        exclude_keys = {"available_markets", "href", "external_ids", "external_urls", "followers", "release_date", "release_date_precision", "total_tracks", "type", "uri", "disc_number", "duration_ms", "explicit", "id", "is_local", "is_playable", "popularity", "preview_url", "track_number", 'images'}
        def clean_dict(d):
            """Recursively remove excluded keys from nested dicts/lists."""
            if isinstance(d, dict):
                return {
                    k: clean_dict(v)
                    for k, v in d.items()
                    if k not in exclude_keys
                }
            elif isinstance(d, list):
                return [clean_dict(i) for i in d]
            else:
                return d
        cleaned_data = clean_dict(data)

        # Pretty print the result
        print(json.dumps(cleaned_data, indent=2))
        return cleaned_data
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

get_spotify_data()

Access Token: BQD389LCBQOOdGOcnjCLPo2Qrac-I8oCvK5S2_vZulqYOQQdB3w3PEtK_kd6fQmw7ngbbmi8KUQ3I8aHV5kIoinSZMxahfxV0hnT-Wi9OBknRFdql0v_b6o1IQuq-Cvmvj5po64O4HE
{
  "tracks": {
    "limit": 5,
    "next": "https://api.spotify.com/v1/search?offset=5&limit=5&query=greedy&type=track",
    "offset": 0,
    "previous": null,
    "total": 6,
    "items": [
      {
        "album": {
          "album_type": "single",
          "artists": [
            {
              "name": "Tate McRae"
            }
          ],
          "name": "greedy"
        },
        "artists": [
          {
            "name": "Tate McRae"
          }
        ],
        "name": "greedy"
      },
      {
        "album": {
          "album_type": "album",
          "artists": [
            {
              "name": "PARTYNEXTDOOR"
            },
            {
              "name": "Drake"
            }
          ],
          "name": "$ome $exy $ongs 4 U"
        },
        "artists": [
          {
            "name": "PARTYNE

{'tracks': {'limit': 5,
  'next': 'https://api.spotify.com/v1/search?offset=5&limit=5&query=greedy&type=track',
  'offset': 0,
  'previous': None,
  'total': 6,
  'items': [{'album': {'album_type': 'single',
     'artists': [{'name': 'Tate McRae'}],
     'name': 'greedy'},
    'artists': [{'name': 'Tate McRae'}],
    'name': 'greedy'},
   {'album': {'album_type': 'album',
     'artists': [{'name': 'PARTYNEXTDOOR'}, {'name': 'Drake'}],
     'name': '$ome $exy $ongs 4 U'},
    'artists': [{'name': 'PARTYNEXTDOOR'}, {'name': 'Drake'}],
    'name': 'GREEDY'},
   {'album': {'album_type': 'album',
     'artists': [{'name': 'Ariana Grande'}],
     'name': 'Dangerous Woman'},
    'artists': [{'name': 'Ariana Grande'}],
    'name': 'Greedy'},
   {'album': {'album_type': 'album',
     'artists': [{'name': 'Tate McRae'}],
     'name': 'THINK LATER'},
    'artists': [{'name': 'Tate McRae'}],
    'name': 'greedy'},
   {'album': {'album_type': 'album',
     'artists': [{'name': 'Tate McRae'}],
     